In [1]:
#Check Python Version
import shutil
import sys
sys.version

'3.8.2 (default, Jan 31 2023, 18:34:03) \n[GCC 12.2.0]'

In [2]:
#Upgrade pip
%pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [3]:
#Check TF & TFX Versioning
import tensorflow as tf
print(tf.__version__)
from tfx import v1 as tfx
print(tfx.__version__)

2023-02-24 14:42:37.497742: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-24 14:42:38.143458: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-24 14:42:38.143528: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-02-24 14:42:40.791407: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

2.11.0
1.12.0


In [4]:
#Setup Variables as examplegen_playground
import os

# Pipeline name
PIPELINE_NAME = "copy_tfrecords"

# Output directory to store artifacts generated from the pipeline.
PIPELINE_ROOT = './artifacts'
# Path to a SQLite DB file to use as an MLMD storage.
METADATA_PATH = os.path.join('metadata', PIPELINE_NAME, 'metadata.db')
# Output directory where created models from the pipeline will be exported.
SERVING_MODEL_DIR = os.path.join('serving_model', PIPELINE_NAME)

# Folder path to data
DATA_ROOT = './data/'

# Folder path to tfrecords
TFRECORDS_TRAIN_DATA_PATH = '../../../../Documents/tfrecord1_split_train/data_tfrecord-00000-of-00001.gz'
TFRECORDS_EVAL_DATA_PATH = '../../../../Documents/tfrecord2_split_eval/data_tfrecord-00000-of-00001.gz'

from absl import logging
logging.set_verbosity(logging.INFO)  # Set default logging level.

In [11]:
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

context = InteractiveContext()

example_gen=tfx.components.CsvExampleGen(
    input_base='./data/'
)

context.run(example_gen)


INFO:absl:Running driver for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:select span and version = (0, None)
INFO:absl:latest span and version = (0, None)
INFO:absl:Running executor for CsvExampleGen
INFO:absl:Generating examples.
INFO:absl:Processing input csv data ./data/* to TFExample.
INFO:absl:Examples generated.
INFO:absl:Running publisher for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

In [6]:
tfrecord_filenames = '../../../../Documents/tfrecord1_split_train/data_tfrecord-00000-of-00001.gz'

# Create a `TFRecordDataset` to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

# Iterate over the first 3 records and decode them.
for tfrecord in dataset.take(3):
  serialized_example = tfrecord.numpy()
  example = tf.train.Example()
  example.ParseFromString(serialized_example)
  print(example)

features {
  feature {
    key: "body_mass_g"
    value {
      float_list {
        value: 0.2916666567325592
      }
    }
  }
  feature {
    key: "culmen_depth_mm"
    value {
      float_list {
        value: 0.6666666865348816
      }
    }
  }
  feature {
    key: "culmen_length_mm"
    value {
      float_list {
        value: 0.2545454502105713
      }
    }
  }
  feature {
    key: "flipper_length_mm"
    value {
      float_list {
        value: 0.1525423675775528
      }
    }
  }
  feature {
    key: "species"
    value {
      int64_list {
        value: 0
      }
    }
  }
}

features {
  feature {
    key: "body_mass_g"
    value {
      float_list {
        value: 0.3055555522441864
      }
    }
  }
  feature {
    key: "culmen_depth_mm"
    value {
      float_list {
        value: 0.511904776096344
      }
    }
  }
  feature {
    key: "culmen_length_mm"
    value {
      float_list {
        value: 0.2690909206867218
      }
    }
  }
  feature {
    key: "flipper

2023-02-24 14:43:42.203925: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-02-24 14:43:42.204061: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-02-24 14:43:42.204126: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (alexanderho1998.c.googlers.com): /proc/driver/nvidia/version does not exist
2023-02-24 14:43:42.205141: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
import json
from typing import Any, Dict, List

from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.v1.types.standard_artifacts import Examples

def _create_pipeline(pipeline_name: str, pipeline_root: str, data_root: str,
                     metadata_path: str) -> tfx.dsl.Pipeline:
  # User input dictionary with split-names and their resepctive uri to tfrecords
  tfrecords_dict={
    "train":TFRECORDS_TRAIN_DATA_PATH,
    "eval":TFRECORDS_EVAL_DATA_PATH
  }
  split_names=''
  counter=1

  # Destination directory for source
  destination_examples_artifact_uri = './examples/'

  # Create a folder in destination_examples_artifact_uri for every key name
  for key in tfrecords_dict:
        split_name_temp = "Split-"+key
        path = os.path.join(destination_examples_artifact_uri, split_name_temp)
        if not os.path.exists(path):
            os.mkdir(path)
        shutil.copy(tfrecords_dict[key], path)

  # Create split_names string
  for key in tfrecords_dict:
      if(counter==1):
          split_names+='["'+key+'","'
          counter=counter+1
      elif(counter==2):
          split_names+=key+'"]'
      else:
          split_names+=key+','
          counter=counter+1

  print(split_names)
  
  # CopyExampleGen execution process to replace Importer Node functionality
  examples_importer = tfx.dsl.Importer(
        source_uri=destination_examples_artifact_uri,
        artifact_type=tfx.types.standard_artifacts.Examples,
        # properties={"split_names": '["train","eva"]'}
        properties={"split_names": split_names}
        ).with_id('examples_importer')

  # Test downstream component
  statistics_gen = tfx.components.StatisticsGen(
     examples=examples_importer.outputs['result'])

  # Following three components will be included in the pipeline.
  components = [
      # example_gen,
      examples_importer,
      statistics_gen
  ]

  return tfx.dsl.Pipeline(
      pipeline_name=pipeline_name,
      pipeline_root=pipeline_root,
      metadata_connection_config=tfx.orchestration.metadata
      .sqlite_metadata_connection_config(metadata_path),
      components=components)

In [10]:
tfx.orchestration.LocalDagRunner().run(
  _create_pipeline(
      pipeline_name=PIPELINE_NAME,
      pipeline_root=PIPELINE_ROOT,
      data_root=DATA_ROOT,
      metadata_path=METADATA_PATH)
  )

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Using deployment config:
 executor_specs {
  key: "StatisticsGen"
  value {
    beam_executable_spec {
      python_executor_spec {
        class_path: "tfx.components.statistics_gen.executor.Executor"
      }
    }
  }
}
metadata_connection_config {
  database_connection_config {
    sqlite {
      filename_uri: "metadata/copy_tfrecords/metadata.db"
      connection_mode: READWRITE_OPENCREATE
    }
  }
}

INFO:absl:Using connection config:
 sqlite {
  filename_uri: "metadata/copy_tfrecords/metadata.db"
  connection_mode: READWRITE_OPENCREATE
}

INFO:absl:Component examples_importer is running.
INFO:absl:Running launcher for node_info {
  type {
    name: "tfx.dsl.components.common.importer.Importer"
  }
  id: "examples_importer"
}
contexts {
  contexts {
    type {
      name: "pipeline"
    }
    name {
      field_value {
        string_value: "copy_tfrecords"
      }
    }
  }
  contexts {
    type {
      n

["train","eval"]


INFO:absl:Statistics for split train written to ./artifacts/StatisticsGen/statistics/72/Split-train.
INFO:absl:Generating statistics for split eval.
INFO:absl:Statistics for split eval written to ./artifacts/StatisticsGen/statistics/72/Split-eval.
INFO:absl:Cleaning up stateless execution info.
INFO:absl:Execution 72 succeeded.
INFO:absl:Cleaning up stateful execution info.
INFO:absl:Publishing output artifacts defaultdict(<class 'list'>, {'statistics': [Artifact(artifact: uri: "./artifacts/StatisticsGen/statistics/72"
, artifact_type: name: "ExampleStatistics"
properties {
  key: "span"
  value: INT
}
properties {
  key: "split_names"
  value: STRING
}
base_type: STATISTICS
)]}) for execution 72
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Component StatisticsGen is finished.


In [17]:
context = InteractiveContext()

# User input dictionary with split-names and their resepctive uri to tfrecords
tfrecords_dict={
    "train":TFRECORDS_TRAIN_DATA_PATH,
    "eval":TFRECORDS_EVAL_DATA_PATH
  }
split_names=''
counter=1

  # Destination directory for source
destination_examples_artifact_uri = './examples/'

  # Create a folder in destination_examples_artifact_uri for every key name
for key in tfrecords_dict:
        split_name_temp = "Split-"+key
        path = os.path.join(destination_examples_artifact_uri, split_name_temp)
        if not os.path.exists(path):
            os.mkdir(path)
        shutil.copy(tfrecords_dict[key], path)

  # Create split_names string
for key in tfrecords_dict:
      if(counter==1):
          split_names+='["'+key+'","'
          counter=counter+1
      elif(counter==2):
          split_names+=key+'"]'
      else:
          split_names+=key+','
          counter=counter+1

print(split_names)

example_gen=tfx.components.CsvExampleGen(
    input_base='./data/'
) 
  
  # CopyExampleGen execution process to replace Importer Node functionality
examples_importer = tfx.dsl.Importer(
        source_uri=destination_examples_artifact_uri,
        artifact_type=tfx.types.standard_artifacts.Examples,
        # properties={"split_names": '["train","eva"]'}
        properties={"split_names": split_names}
        ).with_id('examples_importer')

context.run(example_gen)
context.run(examples_importer)

print("CsvExampleGen Examples Artifact URI: " + example_gen.outputs['examples'].get()[0].uri)
print("Importer Examples Artifact URI: " + examples_importer.outputs['result'].get()[0].uri)

#   Get the URI of the output artifact representing the transformed examples, which is a directory
train_uri = os.path.join(example_gen.outputs['examples'].get()[0].uri, 'Split-train')

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                        for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

# Iterate over the first 3 records and decode them.
for tfrecord in dataset.take(3):
    serialized_example = tfrecord.numpy()
    example = tf.train.Example()
    example.ParseFromString(serialized_example)
    print(example)

INFO:absl:Running driver for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:select span and version = (0, None)
INFO:absl:latest span and version = (0, None)
INFO:absl:Running executor for CsvExampleGen
INFO:absl:Generating examples.
INFO:absl:Processing input csv data ./data/* to TFExample.


["train","eval"]


INFO:absl:Examples generated.
INFO:absl:Running publisher for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running driver for examples_importer
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Processing source uri: ./examples/, properties: {'split_names': '["train","eval"]'}, custom_properties: {}
INFO:absl:Running executor for examples_importer
INFO:absl:Running publisher for examples_importer
INFO:absl:MetadataStore with DB connection initialized


CsvExampleGen Examples Artifact URI: /tmp/tfx-interactive-2023-02-24T14_47_40.372479-fkjpb9ci/CsvExampleGen/examples/1
Importer Examples Artifact URI: ./examples/
features {
  feature {
    key: "body_mass_g"
    value {
      float_list {
        value: 0.2916666567325592
      }
    }
  }
  feature {
    key: "culmen_depth_mm"
    value {
      float_list {
        value: 0.6666666865348816
      }
    }
  }
  feature {
    key: "culmen_length_mm"
    value {
      float_list {
        value: 0.2545454502105713
      }
    }
  }
  feature {
    key: "flipper_length_mm"
    value {
      float_list {
        value: 0.1525423675775528
      }
    }
  }
  feature {
    key: "species"
    value {
      int64_list {
        value: 0
      }
    }
  }
}

features {
  feature {
    key: "body_mass_g"
    value {
      float_list {
        value: 0.3055555522441864
      }
    }
  }
  feature {
    key: "culmen_depth_mm"
    value {
      float_list {
        value: 0.511904776096344
      }

In [ ]:
import re

dict_output = {
	"name": "JSONDecodeError",
	"message": "Expecting value: line 1 column 1 (char 0)",
	"stack": "\u001b[0;31m---------------------------------------------------------------------------\u001b[0m\n\u001b[0;31mJSONDecodeError\u001b[0m                           Traceback (most recent call last)\n\u001b[0;32m/tmp/ipykernel_3107170/380543175.py\u001b[0m in \u001b[0;36m<cell line: 17>\u001b[0;34m()\u001b[0m\n\u001b[1;32m     15\u001b[0m     examples=examples_importer.outputs['result'])\n\u001b[1;32m     16\u001b[0m \u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0;32m---> 17\u001b[0;31m \u001b[0mcontext\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mrun\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0mstatistics_gen\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0m\n\u001b[0;32m~/.pyenv/versions/3.8.2/envs/tfx-3.8.2/lib/python3.8/site-packages/tfx/orchestration/experimental/interactive/notebook_utils.py\u001b[0m in \u001b[0;36mrun_if_ipython\u001b[0;34m(*args, **kwargs)\u001b[0m\n\u001b[1;32m     29\u001b[0m       \u001b[0;31m# __IPYTHON__ variable is set by IPython, see\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m     30\u001b[0m       \u001b[0;31m# https://ipython.org/ipython-doc/rel-0.10.2/html/interactive/reference.html#embedding-ipython.\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0;32m---> 31\u001b[0;31m       \u001b[0;32mreturn\u001b[0m \u001b[0mfn\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0;34m*\u001b[0m\u001b[0margs\u001b[0m\u001b[0;34m,\u001b[0m \u001b[0;34m**\u001b[0m\u001b[0mkwargs\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0m\u001b[1;32m     32\u001b[0m     \u001b[0;32melse\u001b[0m\u001b[0;34m:\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m     33\u001b[0m       logging.warning(\n\n\u001b[0;32m~/.pyenv/versions/3.8.2/envs/tfx-3.8.2/lib/python3.8/site-packages/tfx/orchestration/experimental/interactive/interactive_context.py\u001b[0m in \u001b[0;36mrun\u001b[0;34m(self, component, enable_cache, beam_pipeline_args)\u001b[0m\n\u001b[1;32m    162\u001b[0m         \u001b[0mtelemetry_utils\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mLABEL_TFX_RUNNER\u001b[0m\u001b[0;34m:\u001b[0m \u001b[0mrunner_label\u001b[0m\u001b[0;34m,\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m    163\u001b[0m     }):\n\u001b[0;32m--> 164\u001b[0;31m       \u001b[0mexecution_id\u001b[0m \u001b[0;34m=\u001b[0m \u001b[0mlauncher\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mlaunch\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mexecution_id\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0m\u001b[1;32m    165\u001b[0m \u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m    166\u001b[0m     return execution_result.ExecutionResult(\n\n\u001b[0;32m~/.pyenv/versions/3.8.2/envs/tfx-3.8.2/lib/python3.8/site-packages/tfx/orchestration/launcher/base_component_launcher.py\u001b[0m in \u001b[0;36mlaunch\u001b[0;34m(self)\u001b[0m\n\u001b[1;32m    204\u001b[0m       \u001b[0;31m# be immutable in this context.\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m    205\u001b[0m       \u001b[0;31m# output_dict can still be changed, specifically properties.\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0;32m--> 206\u001b[0;31m       self._run_executor(execution_decision.execution_id,\n\u001b[0m\u001b[1;32m    207\u001b[0m                          \u001b[0mcopy\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mdeepcopy\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0mexecution_decision\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0minput_dict\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m,\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m    208\u001b[0m                          \u001b[0mexecution_decision\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0moutput_dict\u001b[0m\u001b[0;34m,\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\n\u001b[0;32m~/.pyenv/versions/3.8.2/envs/tfx-3.8.2/lib/python3.8/site-packages/tfx/orchestration/launcher/in_process_component_launcher.py\u001b[0m in \u001b[0;36m_run_executor\u001b[0;34m(self, execution_id, input_dict, output_dict, exec_properties)\u001b[0m\n\u001b[1;32m     71\u001b[0m     \u001b[0;31m# be immutable in this context.\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m     72\u001b[0m     \u001b[0;31m# output_dict can still be changed, specifically properties.\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0;32m---> 73\u001b[0;31m     executor.Do(\n\u001b[0m\u001b[1;32m     74\u001b[0m         copy.deepcopy(input_dict), output_dict, copy.deepcopy(exec_properties))\n\n\u001b[0;32m~/.pyenv/versions/3.8.2/envs/tfx-3.8.2/lib/python3.8/site-packages/tfx/components/statistics_gen/executor.py\u001b[0m in \u001b[0;36mDo\u001b[0;34m(self, input_dict, output_dict, exec_properties)\u001b[0m\n\u001b[1;32m     88\u001b[0m     examples = artifact_utils.get_single_instance(\n\u001b[1;32m     89\u001b[0m         input_dict[standard_component_specs.EXAMPLES_KEY])\n\u001b[0;32m---> 90\u001b[0;31m     examples_split_names = artifact_utils.decode_split_names(\n\u001b[0m\u001b[1;32m     91\u001b[0m         examples.split_names)\n\u001b[1;32m     92\u001b[0m     split_names = [\n\n\u001b[0;32m~/.pyenv/versions/3.8.2/envs/tfx-3.8.2/lib/python3.8/site-packages/tfx/types/artifact_utils.py\u001b[0m in \u001b[0;36mdecode_split_names\u001b[0;34m(split_names)\u001b[0m\n\u001b[1;32m    209\u001b[0m   \u001b[0;32mif\u001b[0m \u001b[0;32mnot\u001b[0m \u001b[0msplit_names\u001b[0m\u001b[0;34m:\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m    210\u001b[0m     \u001b[0;32mreturn\u001b[0m \u001b[0;34m[\u001b[0m\u001b[0;34m]\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0;32m--> 211\u001b[0;31m   \u001b[0;32mreturn\u001b[0m \u001b[0mjson\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mloads\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0msplit_names\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0m\u001b[1;32m    212\u001b[0m \u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m    213\u001b[0m \u001b[0;34m\u001b[0m\u001b[0m\n\n\u001b[0;32m~/.pyenv/versions/3.8.2/lib/python3.8/json/__init__.py\u001b[0m in \u001b[0;36mloads\u001b[0;34m(s, cls, object_hook, parse_float, parse_int, parse_constant, object_pairs_hook, **kw)\u001b[0m\n\u001b[1;32m    355\u001b[0m             \u001b[0mparse_int\u001b[0m \u001b[0;32mis\u001b[0m \u001b[0;32mNone\u001b[0m \u001b[0;32mand\u001b[0m \u001b[0mparse_float\u001b[0m \u001b[0;32mis\u001b[0m \u001b[0;32mNone\u001b[0m \u001b[0;32mand\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m    356\u001b[0m             parse_constant is None and object_pairs_hook is None and not kw):\n\u001b[0;32m--> 357\u001b[0;31m         \u001b[0;32mreturn\u001b[0m \u001b[0m_default_decoder\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mdecode\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0ms\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0m\u001b[1;32m    358\u001b[0m     \u001b[0;32mif\u001b[0m \u001b[0mcls\u001b[0m \u001b[0;32mis\u001b[0m \u001b[0;32mNone\u001b[0m\u001b[0;34m:\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m    359\u001b[0m         \u001b[0mcls\u001b[0m \u001b[0;34m=\u001b[0m \u001b[0mJSONDecoder\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\n\u001b[0;32m~/.pyenv/versions/3.8.2/lib/python3.8/json/decoder.py\u001b[0m in \u001b[0;36mdecode\u001b[0;34m(self, s, _w)\u001b[0m\n\u001b[1;32m    335\u001b[0m \u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m    336\u001b[0m         \"\"\"\n\u001b[0;32m--> 337\u001b[0;31m         \u001b[0mobj\u001b[0m\u001b[0;34m,\u001b[0m \u001b[0mend\u001b[0m \u001b[0;34m=\u001b[0m \u001b[0mself\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mraw_decode\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0ms\u001b[0m\u001b[0;34m,\u001b[0m \u001b[0midx\u001b[0m\u001b[0;34m=\u001b[0m\u001b[0m_w\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0ms\u001b[0m\u001b[0;34m,\u001b[0m \u001b[0;36m0\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mend\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0m\u001b[1;32m    338\u001b[0m         \u001b[0mend\u001b[0m \u001b[0;34m=\u001b[0m \u001b[0m_w\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0ms\u001b[0m\u001b[0;34m,\u001b[0m \u001b[0mend\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mend\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m    339\u001b[0m         \u001b[0;32mif\u001b[0m \u001b[0mend\u001b[0m \u001b[0;34m!=\u001b[0m \u001b[0mlen\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0ms\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m:\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\n\u001b[0;32m~/.pyenv/versions/3.8.2/lib/python3.8/json/decoder.py\u001b[0m in \u001b[0;36mraw_decode\u001b[0;34m(self, s, idx)\u001b[0m\n\u001b[1;32m    353\u001b[0m             \u001b[0mobj\u001b[0m\u001b[0;34m,\u001b[0m \u001b[0mend\u001b[0m \u001b[0;34m=\u001b[0m \u001b[0mself\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mscan_once\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0ms\u001b[0m\u001b[0;34m,\u001b[0m \u001b[0midx\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m    354\u001b[0m         \u001b[0;32mexcept\u001b[0m \u001b[0mStopIteration\u001b[0m \u001b[0;32mas\u001b[0m \u001b[0merr\u001b[0m\u001b[0;34m:\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0;32m--> 355\u001b[0;31m             \u001b[0;32mraise\u001b[0m \u001b[0mJSONDecodeError\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0;34m\"Expecting value\"\u001b[0m\u001b[0;34m,\u001b[0m \u001b[0ms\u001b[0m\u001b[0;34m,\u001b[0m \u001b[0merr\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mvalue\u001b[0m\u001b[0;34m)\u001b[0m \u001b[0;32mfrom\u001b[0m \u001b[0;32mNone\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0m\u001b[1;32m    356\u001b[0m         \u001b[0;32mreturn\u001b[0m \u001b[0mobj\u001b[0m\u001b[0;34m,\u001b[0m \u001b[0mend\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\n\u001b[0;31mJSONDecodeError\u001b[0m: Expecting value: line 1 column 1 (char 0)"
}

ansi_escape = re.compile(r'\x1B(?:[@-Z\\-_]|\[[0-?]*[ -/]*[@-~])')
result = ansi_escape.sub('', dict_output["stack"])

print(result)